In [12]:
import gmsh


def is_simple(step_path):
    gmsh.clear()

    gmsh.model.add("mesh_volume")
    gmsh.model.occ.importShapes(str(step_path))
    gmsh.model.occ.synchronize()

    return len(gmsh.model.getEntities(3)) == 1

In [ ]:
from pathlib import Path
import shutil
import trimesh


abc_path = Path("meshes/abc")
raw_step_path = abc_path / "abc_step"
raw_stl_path = abc_path / "abc_stl"
step_path = abc_path / "step"
stl_path = abc_path / "stl"
step_path.mkdir(parents=True, exist_ok=True)
stl_path.mkdir(parents=True, exist_ok=True)


count = 0
gmsh.initialize()
gmsh.option.setNumber("General.Verbosity", 1)
for i in range(100):
    stl_subdir = raw_stl_path / f"{i:08d}"
    step_subdir = raw_step_path / f"{i:08d}"

    stl_files = [p for p in stl_subdir.iterdir() if p.is_file()]
    step_files = [p for p in step_subdir.iterdir() if p.is_file()]
    if not stl_files or not step_files:
        continue

    stl_file = stl_files[0]
    step_file = step_files[0]
    stl_out_path = stl_path / f"{stl_subdir.name}.stl"
    step_out_path = step_path / f"{step_subdir.name}.step"
    mesh = trimesh.load_mesh(stl_file)
    if len(mesh.vertices) <= 1000 and is_simple(step_file):
        shutil.copy2(step_file, step_out_path)
        shutil.copy2(stl_file, stl_out_path)
        count += 1


print(f"Found {count} simple meshes.")
gmsh.finalize()


Found 9 simple meshes.
